<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/selectivity_2_HL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
torch.manual_seed(1234)
np.random.seed(1234)

In [4]:
mnist_trainset = datasets.MNIST(root='./data', train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = datasets.MNIST(root='./data', 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=50, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

In [5]:
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        # modify this section for later use 
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 256)
        self.linear_3 = torch.nn.Linear(256, 10)
        self.sigmoid12  = torch.nn.Sigmoid()
        self.sigmoid23  = torch.nn.Sigmoid()

        self.layer_activations = dict()

    def forward(self, x):

        # modify this section for later use 
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid12(x)
        x = self.linear_2(x)
        x = self.sigmoid23(x)
        pred = self.linear_3(x)
        return pred

In [6]:
def get_activation(model, layer_name):    
    def hook(module, input, output):
        model.layer_activations[layer_name] = output
    return hook

In [7]:
def selectivity(hidden_layer_each_neuron):
    __selectivity__ = list()
    # I will now try to find the average of each class for each neuron.
    # check out the next cell 
    avg_activations = [dict() for x in range(256)]
    for i, neuron in enumerate(hidden_layer_each_neuron):
        for k, v in neuron.items():
            # v is the list of activations for hidden layer's neuron k 
            avg_activations[i][k] = sum(v) / float(len(v))

    # generate 256 lists to get only values in avg_activations
    only_activation_vals = [list() for x in range(256)]

    # selectivity_list contains all of the selectivity of each neuron 
    selectivity = list()

    # get only values from avg_activations
    for i, avg_activation in enumerate(avg_activations):
        for value in avg_activation.values():
            only_activation_vals[i].append(value)


    for activation_val in only_activation_vals:
        # find u_max 
        u_max = np.max(activation_val)

        # find u_minus_max 
        u_minus_max = (np.sum(activation_val) - u_max) / 9

        # find selectivity 
        selectivity = (u_max - u_minus_max) / (u_max + u_minus_max)

        # append selectivity value to selectivity
        __selectivity__.append(selectivity)

    avg_selectivity = np.average(__selectivity__)
    std_selectivity = np.std(__selectivity__)
                                 
    return avg_selectivity, std_selectivity

In [8]:
def model_factory(optimizer_name):
    '''
    optimizer_name : choose one of Adagrad, Adadelta, SGD, and Adam 

    '''
    my_model = Model()
    print("my_model:", my_model)
    my_model.to(device)

    # modify this section for later use 
    my_model.sigmoid12.register_forward_hook(get_activation(my_model, 's12'))
    my_model.sigmoid23.register_forward_hook(get_activation(my_model, 's23'))

    if optimizer_name == 'Adadelta':
        my_optimizer = torch.optim.Adadelta(my_model.parameters(), lr=1.0)

    elif optimizer_name == 'Adagrad':
        my_optimizer = torch.optim.Adagrad(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'SGD':
        my_optimizer = torch.optim.SGD(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'Adam':
        my_optimizer = torch.optim.Adam(my_model.parameters(), lr=0.001)

    else:
        print("ERROR")
    
    print("my_optimizer:", my_optimizer)
    test_acc, selectivity_list_avg, selectivity_list_std = selectivity_trainer(optimizer=my_optimizer, model=my_model)
    file_saver = open(f"2HL_selectivity_{optimizer_name}.txt", "w")
    file_saver.write(str(test_acc)+'\n'+str(selectivity_list_avg)+'\n'+str(selectivity_list_std)+'\n\n')
    file_saver.close()

    if optimizer_name == 'Adadelta':
        !cp 2HL_selectivity_Adadelta.txt /content/drive/MyDrive
    
    elif optimizer_name == 'Adagrad':
        !cp 2HL_selectivity_Adagrad.txt /content/drive/MyDrive

    elif optimizer_name == 'SGD':
        !cp 2HL_selectivity_SGD.txt /content/drive/MyDrive

    elif optimizer_name == 'Adam':
        !cp 2HL_selectivity_Adam.txt /content/drive/MyDrive

    else:
        print("ERROR")

In [9]:
def avg_std_calculator(_hidden_layer_each_neuron_12, _hidden_layer_each_neuron_23):

    avg_selectivity12, std_selectivity12 = selectivity(_hidden_layer_each_neuron_12)
    avg_selectivity23, std_selectivity23 = selectivity(_hidden_layer_each_neuron_23)

    final_selectivity_avg = (avg_selectivity12 + avg_selectivity23) / 2 
    final_selecvitity_std = (std_selectivity12 + std_selectivity23) / 2 

    return final_selectivity_avg, final_selecvitity_std

In [10]:
no_epochs = 30
def selectivity_trainer(optimizer, model):

    criterion = torch.nn.CrossEntropyLoss()
    train_loss = list()
    test_loss  = list()
    test_acc   = list()

    best_test_loss = 1

    selectivity_avg_list = list()
    selectivity_std_list = list()

    for epoch in range(no_epochs):

        # modify this section for later use 
        hidden_layer_each_neuron_12 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_12 = np.array(hidden_layer_each_neuron_12)

        hidden_layer_each_neuron_23 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_23 = np.array(hidden_layer_each_neuron_23)
        # modify this section for later use 


        total_train_loss = 0
        total_test_loss = 0

        # training
        # set up training mode 
        model.train()

        for itr, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            pred = model(images)

            loss = criterion(pred, labels)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if labels[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            
            for activation, label in zip(model.layer_activations['s12'], labels):
                # shape of activation and label: 256 and 1 
                
                # get the actual value of item. This is because label is now Tensor 
                label = label.item()

                # this is not part of gradient calculcation 
                with torch.no_grad():
                    activation = activation.numpy()

                # for each image/label, append activation value of neuron 
                for i in range(256):    # number of neurons in hidden layer 
                    hidden_layer_each_neuron_12[i][label].append(activation[i])

            for activation, label in zip(model.layer_activations['s23'], labels):
                # shape of activation and label: 256 and 1 
                
                # get the actual value of item. This is because label is now Tensor 
                label = label.item()

                # this is not part of gradient calculcation 
                with torch.no_grad():
                    activation = activation.numpy()

                # for each image/label, append activation value of neuron 
                for i in range(256):    # number of neurons in hidden layer 
                    hidden_layer_each_neuron_23[i][label].append(activation[i])

        selectivity_avg, selecvitity_std = avg_std_calculator(hidden_layer_each_neuron_12, hidden_layer_each_neuron_23)
        
        selectivity_avg_list.append(selectivity_avg)
        selectivity_std_list.append(selecvitity_std)

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))


    return test_acc, selectivity_avg_list, selectivity_std_list

# Adadelta

In [11]:
model_factory('Adadelta')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 0.66955220, Test Loss: 0.29091561, Test Accuracy: 0.91560000

Epoch: 2/30, Train Loss: 0.23864885, Test Loss: 0.17893777, Test Accuracy: 0.94570000

Epoch: 3/30, Train Loss: 0.16415371, Test Loss: 0.14251487, Test Accuracy: 0.95830000

Epoch: 4/30, Train Loss: 0.12487799, Test Loss: 0.10738104, Test Accuracy: 0.96570000

Epoch: 5/30, Train Loss: 0.09811718, Test Loss: 0.09940851, Test Accuracy: 0.96730000

Epoch: 6/30, Train Loss: 0.08101228, Test Loss: 0.09228468, Test Accuracy: 0.97040000

Epoch: 7/30, Train Loss: 0.06785859, Test Loss: 0.08866793, Test Accuracy: 0.97140000

Epoch: 8/30, Tra

# AdaGrad

In [12]:
model_factory('Adagrad')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
)
my_optimizer: Adagrad (
Parameter Group 0
    eps: 1e-10
    initial_accumulator_value: 0
    lr: 0.1
    lr_decay: 0
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 0.42294847, Test Loss: 0.11472658, Test Accuracy: 0.96540000

Epoch: 2/30, Train Loss: 0.08807486, Test Loss: 0.09191858, Test Accuracy: 0.97240000

Epoch: 3/30, Train Loss: 0.05717871, Test Loss: 0.08132470, Test Accuracy: 0.97510000

Epoch: 4/30, Train Loss: 0.03934799, Test Loss: 0.07855366, Test Accuracy: 0.97780000

Epoch: 5/30, Train Loss: 0.02771339, Test Loss: 0.07289141, Test Accuracy: 0.97870000

Epoch: 6/30, Train Loss: 0.01995135, Test Loss: 0.07290223, Test Accuracy: 0.97900000

Epoch: 7/30, Train Loss: 0.01444415, Test Loss: 0.07167542, Test Acc

# SGD


In [13]:
model_factory('SGD')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
)
my_optimizer: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 1.70086394, Test Loss: 0.77447292, Test Accuracy: 0.74410000

Epoch: 2/30, Train Loss: 0.55363856, Test Loss: 0.41966936, Test Accuracy: 0.87930000

Epoch: 3/30, Train Loss: 0.39161087, Test Loss: 0.34878524, Test Accuracy: 0.89910000

Epoch: 4/30, Train Loss: 0.34210257, Test Loss: 0.30893291, Test Accuracy: 0.91020000

Epoch: 5/30, Train Loss: 0.30959184, Test Loss: 0.29351307, Test Accuracy: 0.91450000

Epoch: 6/30, Train Loss: 0.28182664, Test Loss: 0.26428919, Test Accuracy: 0.92290000

Epoch: 7/30, Train Loss: 0.25754871, Test Loss: 0.24462154, Test Accuracy: 0.928700

# Adam

In [14]:
model_factory('Adam')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
)
my_optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 0.45788566, Test Loss: 0.19843138, Test Accuracy: 0.94080000

Epoch: 2/30, Train Loss: 0.16914621, Test Loss: 0.14410988, Test Accuracy: 0.95550000

Epoch: 3/30, Train Loss: 0.11548502, Test Loss: 0.11379071, Test Accuracy: 0.96530000

Epoch: 4/30, Train Loss: 0.08344443, Test Loss: 0.08751983, Test Accuracy: 0.97290000

Epoch: 5/30, Train Loss: 0.06500032, Test Loss: 0.08379442, Test Accuracy: 0.97350000

Epoch: 6/30, Train Loss: 0.04957014, Test Loss: 0.09894467, Test Accuracy: 0.96930000

Epoch: 7/30, Train Loss: 0.03961783, Test Loss: 0.08257195, Test Accuracy: 